In [27]:
import pandas as pd
import string, re
from sklearn.model_selection import train_test_split

import sklearn_crfsuite
from sklearn_crfsuite import  metrics

import pickle

In [28]:
sheet_names = ['1', '2', '4']
data = pd.Series()

for s in sheet_names:
    x = pd.read_excel('entity_ext.xlsx', engine='openpyxl',
                      sheet_name=s, usecols='C', header=None, dtype=str)
    print(len(x))
    # print(x[2])
    data = data.append(x[2].dropna())

data

d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


18609
18610
18611


0        Salam men [albali kartin](card_type) limiti ar...
1                                                     Beli
2        <last_name> <first_name> <first_name> oglu <mo...
3                         Yaxsi coxsagolun tewekkur edirem
5                                                    Salam
                               ...                        
18606            Yeni [bir ilden sora](date) götürmek olar
18607    Xanim sizi yoruram lap deqiwlesdirmek istiyire...
18608    Salam [Albalinin](card_type) limitini artirmaq...
18609    Salam mende [albali plus](card_type) varimdi l...
18610    Salam [albali plus](card_type) limiti artirmaq...
Length: 53933, dtype: object

In [29]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
    count = 0

    for _, sentence in tagged_sentences.iteritems():

        # print(sentence)
        if len(sentence) < 2:
            continue

        count += 1

        # words = re.findall(r"\[(.*?)]", sentence)
        # tags = re.findall(r"\((.*?)\)", sentence)

        words = []
        tags = []
        i = j = 0

        try:
            while True:
                sentence = sentence[j:]
                i = sentence.index('[')
                j = sentence.index(']')

                if sentence[j + 1] != '(':  # no tag
                    raise

                words.append(sentence[i + 1:j])
                sentence = sentence[j + 1:]

                j = sentence.index(')')
                tags.append(sentence[1:j])

        except:
            pass

        # make sure their length are equal
        if len(words) != len(tags):
            print(_)
            print(sentence)
            print(words, tags)

        # print(w)
        X.append(words)
        y.append(tags)

    return X, y


sents, tags = transform_to_dataset(data)
print(list(set(sum(tags, []))))

['name', 'app', 'loc', 'patronymic', 'card_type', 'surname', 'term', 'currency', 'date', 'amount', 'credit_type', 'weekdays', 'time', 'org', 'email', 'p_amount', 'Ot', 'o_system', 'deposit_type', 'pay_dev', 'phone']


In [30]:
def word2features(sent, i):
    word = sent[i]

    features = {
        'bias': 1.0,
        'word': word,
        'len(word)': len(word),
        'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-4:]': word[-4:],
        'word.lower()': word.lower(),
        'word.ispunctuation': (word in string.punctuation),
        'word.isdigit()': word.isdigit(),
    }
    if i > 0:
        word1 = sent[i-1]
        features.update({
            '-1:word': word1,
            '-1:len(word)': len(word1),
            '-1:word.lower()': word1.lower(),
            '-1:word[:3]': word1[:3],
            '-1:word[:2]': word1[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.ispunctuation': (word1 in string.punctuation),
        })
    else:
        features['BOS'] = True
    
    if i > 1:
        word2 = sent[i-2]
        features.update({
            '-2:word': word2,
            '-2:len(word)': len(word2),
            '-2:word.lower()': word2.lower(),
            '-2:word[:3]': word2[:3],
            '-2:word[:2]': word2[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.ispunctuation': (word2 in string.punctuation),
        })

    if i < len(sent)-1:
        word1 = sent[i+1]
        features.update({
            '+1:word': word1,
            '+1:len(word)': len(word1),
            '+1:word.lower()': word1.lower(),
            '+1:word[:3]': word1[:3],
            '+1:word[:2]': word1[:2],
            '+1:word[-3:]': word1[-3:],
            '+1:word[-2:]': word1[-2:],
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.ispunctuation': (word1 in string.punctuation),
        })

    else:
        features['EOS'] = True
    if i < len(sent) - 2:
        word2 = sent[i+2]
        features.update({
            '+2:word': word2,
            '+2:len(word)': len(word2),
            '+2:word.lower()': word2.lower(),
            '+2:word[:3]': word2[:3],
            '+2:word[:2]': word2[:2],
            '+2:word[-3:]': word2[-3:],
            '+2:word[-2:]': word2[-2:],
            '+2:word.isdigit()': word2.isdigit(),
            '+2:word.ispunctuation': (word2 in string.punctuation),
        })

    return features
  
def sent2features(sent):
  return [word2features(sent, i) for i in range(len(sent))]


In [31]:
trainDataX, testDataX, trainDatay, testDatay = train_test_split(sents, tags, test_size=0.10, random_state=42)

In [32]:
Xtrain = [sent2features(s) for s in trainDataX]
ytrain = trainDatay

Xtest = [sent2features(s) for s in testDataX]
ytest = testDatay

In [33]:
# Xtrain

In [34]:
crf = sklearn_crfsuite.CRF(
    algorithm = 'lbfgs',
    c1 = 0.25,
    c2 = 0.3,
    max_iterations = 100,
    all_possible_transitions=True
)

print(len(Xtrain), len(ytrain))
crf.fit(Xtrain, ytrain)                  
#training the model

47735 47735


d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.25, c2=0.3,
    max_iterations=100)

In [35]:
labels = list(crf.classes_)

ypred = crf.predict(Xtrain)
print('F1 score on the train set = {}\n'.format(metrics.flat_f1_score(ytrain, ypred, average='weighted', labels=labels)))
print('Accuracy on the train set = {}\n'.format(metrics.flat_accuracy_score(ytrain, ypred)))

# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
print('Train set classification report: \n\n{}'.format(metrics.flat_classification_report(
ytrain, ypred, labels, digits=3
)))
#obtaining metrics such as accuracy, etc. on the test set
ypred = crf.predict(Xtest)
print('F1 score on the test set = {}\n'.format(metrics.flat_f1_score(ytest, ypred,
average='weighted', labels=labels)))
print('Accuracy on the test set = {}\n'.format(metrics.flat_accuracy_score(ytest, ypred)))

# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
print('Test set classification report: \n\n{}'.format(metrics.flat_classification_report(ytest, ypred, labels, digits=3)))

F1 score on the train set = 0.9937483836151744

Accuracy on the train set = 0.993758387167244



d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=['surname', 'currency', 'date', 'card_type', 'amount', 'loc', 'pay_dev', 'org', 'Ot', 'time', 'name', 'term', 'app', 'credit_type', 'weekdays', 'phone', 'p_amount', 'patronymic', 'deposit_type', 'email', 'o_system'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


Train set classification report: 

              precision    recall  f1-score   support

     surname      0.987     1.000     0.993       147
    currency      0.998     0.998     0.998      4151
        date      0.995     0.989     0.992      2216
   card_type      0.989     0.997     0.993      5270
      amount      0.998     0.997     0.997      4297
         loc      0.982     0.976     0.979       802
     pay_dev      0.978     0.997     0.987       310
         org      0.986     0.990     0.988      2213
          Ot      0.999     0.998     0.999      7655
        time      0.993     0.987     0.990       703
        name      0.983     0.994     0.988       172
        term      0.984     0.991     0.988      1442
         app      0.986     0.964     0.975      1135
 credit_type      0.996     0.963     0.979       267
    weekdays      0.993     1.000     0.996       268
       phone      0.993     0.997     0.995       293
    p_amount      0.998     0.996     0.997   

d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\sklearn\metrics\_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\onedrive - ada university\projects\python\freelancer\venv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

In [36]:

filename = 'model/crf.sav'
f = open(filename, 'wb')
pickle.dump(crf, f)
f.close()